In [1]:
import numpy as np
import pandas as pd

In [2]:
pred = pd.DataFrame()
pred['srch_id'] = [1,1,1,1,1,2,2,2,3,3,3,3,3,3,3]
pred['prop_id'] = [1,2,3,5,4,1,2,3,7,6,5,4,3,2,1]

truth = pd.DataFrame()
truth['srch_id'] = [1,1,1,1,1,2,2,2,3,3,3,3,3,3,3]
truth['prop_id'] = [1,2,3,4,5,1,2,3,1,2,3,4,5,6,7]
truth['booking_bool'] = [1,0,0,0,0,1,0,0,1,0,0,0,0,0,0]
truth['click_bool'] = [1,1,1,1,0,1,1,0,1,1,1,0,0,0,0]

In [58]:
def ideal_ranking(ground_truth):
    ideal_ranking = ground_truth.sort_values(by=['booking_bool', 'click_bool'], ascending=False)
    return ideal_ranking.reset_index(drop=True)
    
def DCG(prediction, ground_truth):
    dcg = []
    for row in prediction.itertuples():
        prop_id = getattr(row, 'prop_id')
        rank = getattr(row, 'Index') + 1
        booked = ground_truth.set_index('prop_id').at[(prop_id,'booking_bool')]
        clicked = ground_truth.set_index('prop_id').at[(prop_id,'click_bool')]
        score = 5 if booked else 1 if clicked else 0
        dcg.append((2**score - 1)/(np.log2(rank + 1)))
    return sum(dcg)

def nDCG(prediction, ground_truth):
    prediction.reset_index(drop=True, inplace=True)
    dcg = DCG(prediction, ground_truth)
    idcg = DCG(ideal_ranking(ground_truth), ground_truth)
    return(dcg/idcg)

def overall_nDCG(prediction, ground_truth):
    ids = prediction.srch_id.unique()
    all_nDCG = [nDCG(prediction[prediction.srch_id==i], ground_truth[ground_truth.srch_id==i]) for i in ids]
    print(all_nDCG)
    print(np.mean(all_nDCG))

In [59]:
overall_nDCG(pred, truth)

[0.9986541281035272, 1.0, 0.3447268228036483]
0.7811269836357252
